## This below code allows selecting direction of your choice
## then it picks up a hedge option and a future 

In [1]:
import ipywidgets as widgets
from IPython.display import display, HTML

def pick_direction(kite, instruments, lp, up):
    """
    Displays Bullish/Bearish strategy buttons, runs the selected strategy,
    and collects futures symbols, option symbols, and tokens.

    Returns:
    - fut_symbols: list
    - option_symbols: list
    - tokens: list
    """

    output = widgets.Output()

    # Global vars you want to collect
    global fut_symbols, option_symbols, tokens
    fut_symbols = []
    option_symbols = []
    tokens = []

    # ------------------------ BULLISH ------------------------
    def on_bullish_click(b):
        global fut_symbols, option_symbols, tokens

        # Your function returns ((symbol, token), (symbol, token))
        (fut_symbol, fut_token), (put_symbol, put_token) = bull_put_spread_with_future(
            kite, instruments, lower_premium=lp, upper_premium=up
        )

        # Store cleaned values
        fut_symbols = [fut_symbol]
        option_symbols = [put_symbol]
        tokens = [fut_token, put_token]

        with output:
            output.clear_output()
            display(HTML(
                "<h3 style='color:green;'>🟢 BULLISH STRATEGY SELECTED — PRICE EXPECTED TO MOVE UP 📈</h3>"
            ))
            print("▶ FUTURES symbol:", fut_symbols)
            print("▶ OPTION hedge symbol (PUT):", option_symbols)
            print("▶ TOKENS:", tokens)

    # ------------------------ BEARISH ------------------------
    def on_bearish_click(b):
        global fut_symbols, option_symbols, tokens

        # Your function returns ((symbol, token), (symbol, token))
        (fut_symbol, fut_token), (call_symbol, call_token) = bear_call_spread_with_future(
            kite, instruments, lower_premium=lp, upper_premium=up
        )

        # Store cleaned values
        fut_symbols = [fut_symbol]
        option_symbols = [call_symbol]
        tokens = [fut_token, call_token]

        with output:
            output.clear_output()
            display(HTML(
                "<h3 style='color:red;'>🔴 BEARISH STRATEGY SELECTED — PRICE EXPECTED TO MOVE DOWN 📉</h3>"
            ))
            print("▶ FUTURES symbol:", fut_symbols)
            print("▶ OPTION hedge symbol (CALL):", option_symbols)
            print("▶ TOKENS:", tokens)

    # ------------------------ BUTTONS ------------------------
    bullish_button = widgets.Button(description="Bullish Strategy", button_style='success')
    bearish_button = widgets.Button(description="Bearish Strategy", button_style='danger')

    bullish_button.on_click(on_bullish_click)
    bearish_button.on_click(on_bearish_click)

    display(widgets.VBox([widgets.HBox([bullish_button, bearish_button]), output]))

    # Return all 3 variables
    return fut_symbols, option_symbols, tokens


# This is the primary margin checking & order placing code

In [ ]:
def generate_qty_options(lot_size):
    return [lot_size * i for i in range(1, 7)]
qty_options = generate_qty_options(lot_size = 75)

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

def make_order_basket(fut_symbols, option_symbols, kite):
    orders = []
    kite_orange = 'yellow'

    # -----------------------------
    # Output Widgets
    # -----------------------------
    basket_output = widgets.Output()   # Shows order basket
    msg_output = widgets.Output()      # Shows margin info, placement messages

    # -----------------------------
    # Quantity Buttons
    # -----------------------------
    def create_qty_buttons():
        selected_qty = qty_options[0]
        buttons = []
        for q in qty_options:
            btn = widgets.Button(description=str(q),
                                 layout=widgets.Layout(width='60px', margin='0 8px 0 0'))
            buttons.append(btn)

        qty_out = widgets.Output()
        qty_box = widgets.VBox([widgets.HBox(buttons + [qty_out])],
                               layout=widgets.Layout(margin='0 0 10px 0'))

        def update_styles(clicked_btn):
            nonlocal selected_qty
            selected_qty = int(clicked_btn.description)
            for b in buttons:
                b.style.button_color = kite_orange if b == clicked_btn else None
            with qty_out:
                clear_output()

        for b in buttons:
            b.on_click(update_styles)

        update_styles(buttons[0])
        return qty_box, lambda: selected_qty

    # -----------------------------
    # Buy/Sell Buttons
    # -----------------------------
    def create_side_buttons(default='BUY'):
        selected_side = default
        buy_btn = widgets.Button(description="BUY",
                                 layout=widgets.Layout(width='60px', margin='0 8px 0 0'))
        sell_btn = widgets.Button(description="SELL",
                                  layout=widgets.Layout(width='60px', margin='0 8px 0 0'))

        side_box = widgets.VBox([widgets.HBox([buy_btn, sell_btn])],
                                layout=widgets.Layout(margin='0 0 10px 0'))

        def update_styles(btn_clicked):
            nonlocal selected_side
            selected_side = btn_clicked.description
            buy_btn.style.button_color = '#2196f3' if selected_side == 'BUY' else '#ccc'
            sell_btn.style.button_color = 'red' if selected_side == 'SELL' else '#ccc'

        buy_btn.on_click(update_styles)
        sell_btn.on_click(update_styles)
        update_styles(widgets.Button(description=selected_side))
        return side_box, lambda: selected_side

    # -----------------------------
    # Order Widget Set with Dynamic Price Fields
    # -----------------------------
    def create_order_widget(symbols_list, default_buy=True):
        tradingsymbol = widgets.Dropdown(options=symbols_list, description="Symbol:",
                                         layout=widgets.Layout(margin='0 0 10px 0'))
        side_box, get_side = create_side_buttons('BUY' if default_buy else 'SELL')
        qty_box, get_qty = create_qty_buttons()
        order_type = widgets.Dropdown(options=["MARKET", "LIMIT","SL", "SL-M"], value="MARKET",
                                      description="Type:", layout=widgets.Layout(margin='0 0 10px 0'))
        
        # Price input fields
        price_input = widgets.FloatText(
            value=0.0,
            description='Price:',
            layout=widgets.Layout(margin='0 0 10px 0')
        )
        
        trigger_price_input = widgets.FloatText(
            value=0.0,
            description='Trigger:',
            layout=widgets.Layout(margin='0 0 10px 0')
        )
        
        # Dynamic price container
        price_container = widgets.VBox([], layout=widgets.Layout(margin='0 0 10px 0'))
        
        # Function to update visible price fields based on order type
        def update_price_fields(change):
            order_type_val = change['new']
            if order_type_val == 'MARKET':
                price_container.children = []
            elif order_type_val == 'LIMIT':
                price_container.children = [price_input]
            elif order_type_val == 'SL':
                price_container.children = [trigger_price_input, price_input]
            elif order_type_val == 'SL-M':
                price_container.children = [trigger_price_input]
        
        order_type.observe(update_price_fields, names='value')
        update_price_fields({'new': order_type.value})
        
        product = widgets.Dropdown(options=["MIS", "CNC", "NRML"], value="MIS",
                                   description="Product:", layout=widgets.Layout(margin='0 0 10px 0'))
        variety = widgets.Dropdown(options=["regular", "amo"], value="regular",
                                   description="Variety:", layout=widgets.Layout(margin='0 0 10px 0'))
        add_btn = widgets.Button(description="Add Order", button_style="primary",
                                 layout=widgets.Layout(margin='0 0 20px 0'))

        return {
            "tradingsymbol": tradingsymbol,
            "get_side": get_side,
            "side_box": side_box,
            "qty_box": qty_box,
            "get_qty": get_qty,
            "order_type": order_type,
            "price_input": price_input,
            "trigger_price_input": trigger_price_input,
            "price_container": price_container,
            "product": product,
            "variety": variety,
            "add_btn": add_btn
        }

    # -----------------------------
    # Determine default side for futures based on option_symbols
    # -----------------------------
    default_fut_side = "BUY"  # fallback
    if option_symbols and isinstance(option_symbols, list):
        first_opt = option_symbols[0]  # take the first option symbol
        if first_opt.endswith("PE"):
            default_fut_side = "BUY"
        elif first_opt.endswith("CE"):
            default_fut_side = "SELL"

    fut_widget = create_order_widget(fut_symbols, default_buy=(default_fut_side=="BUY"))
    opt_widget = create_order_widget(option_symbols, default_buy=True)

    # -----------------------------
    # Add Order Logic
    # -----------------------------
    def add_order_widget(widget):
        sym = widget["tradingsymbol"].value
        txn_type = widget["get_side"]()
        qty = widget["get_qty"]()
        order_type_val = widget["order_type"].value

        if sym.endswith(("CE", "PE")):
            txn_type = "BUY"

        order = {
            "tradingsymbol": sym,
            "exchange": "NFO",
            "transaction_type": txn_type,
            "quantity": qty,
            "order_type": order_type_val,
            "product": widget["product"].value,
            "variety": widget["variety"].value
        }
        
        # Add price fields based on order type
        if order_type_val == 'LIMIT':
            order['price'] = widget['price_input'].value
        elif order_type_val == 'SL':
            order['trigger_price'] = widget['trigger_price_input'].value
            order['price'] = widget['price_input'].value
        elif order_type_val == 'SL-M':
            order['trigger_price'] = widget['trigger_price_input'].value

        orders.append(order)

        with basket_output:
            clear_output()
            display(HTML("<span style='color: orange; font-weight: bold;'>🛒 Order Basket:</span>"))
            for o in orders:
                side_color = "green" if o["transaction_type"] == "BUY" else "red"
                box_style = (
                    "display: inline-block; padding: 5px 10px; margin: 2px 4px; "
                    "border-radius: 5px; font-family: monospace; font-weight: bold; color: white;"
                )
                symbol_box = f"<span style='{box_style} background-color: #2196f3;'>{o['tradingsymbol']}</span>"
                qty_box_html = f"<span style='{box_style} background-color: #2196f3;'>{o['quantity']}</span>"
                side_box_html = f"<span style='{box_style} background-color: {side_color};'>{o['transaction_type']}</span>"
                type_box = f"<span style='{box_style} background-color: #2196f3;'>{o['order_type']}</span>"
                
                # Show price info if available
                price_info = ""
                if 'price' in o:
                    price_info += f" @ ₹{o['price']:.2f}"
                if 'trigger_price' in o:
                    price_info += f" (Trigger: ₹{o['trigger_price']:.2f})"
                
                product_box = f"<span style='{box_style} background-color: #2196f3;'>{o['product']}</span>"
                variety_box = f"<span style='{box_style} background-color: #2196f3;'>{o['variety']}</span>"
                
                display(HTML(symbol_box + qty_box_html + side_box_html + type_box + 
                           f"<span style='color: #666; margin: 0 5px;'>{price_info}</span>" +
                           product_box + variety_box))

    fut_widget["add_btn"].on_click(lambda b: add_order_widget(fut_widget))
    opt_widget["add_btn"].on_click(lambda b: add_order_widget(opt_widget))

    # -----------------------------
    # Clear Orders
    # -----------------------------
    clear_btn = widgets.Button(description="Clear Orders", button_style="danger")
    def clear_orders(b):
        orders.clear()
        with basket_output:
            clear_output()
            print("🗑️ All orders cleared.")
        with msg_output:
            clear_output()
    clear_btn.on_click(clear_orders)

    # -----------------------------
    # Check Margin
    # -----------------------------
    margin_btn = widgets.Button(description="Check Margin", button_style="info")
    def check_margin_required():
        if not orders:
            with msg_output:
                clear_output()
                print("⚠️ No orders to check margin for!")
            return None, None

        margin_orders = []
        for o in orders:
            ex = "NFO"
            margin_order = {
                "exchange": ex,
                "tradingsymbol": o["tradingsymbol"],
                "transaction_type": o["transaction_type"],
                "quantity": o["quantity"],
                "product": o["product"],
                "order_type": o["order_type"]
            }
            # Add price fields if they exist
            if 'price' in o:
                margin_order['price'] = o['price']
            if 'trigger_price' in o:
                margin_order['trigger_price'] = o['trigger_price']
            
            margin_orders.append(margin_order)

        try:
            margins = kite.margins()
            margin_info = kite.basket_order_margins(margin_orders, consider_positions=True, mode="compact")
            available_balance = margins['equity']['net']
            df = pd.json_normalize(margin_info['orders'], sep='_')
            total_required = df['total'].sum()

            with msg_output:
                clear_output()
                print(f"💰 Available Balance: ₹{available_balance:,.2f}")
                print(f"📊 Margin Required: ₹{total_required:,.2f}")
                if total_required > available_balance:
                    print("⚠️ Insufficient funds to place these orders!")
                else:
                    print("✅ Sufficient funds available.")

            return total_required, available_balance
        except Exception as e:
            with msg_output:
                clear_output()
                print(f"❌ Error checking margin: {e}")
            return None, None

    margin_btn.on_click(lambda b: check_margin_required())

    # -----------------------------
    # Internal function to execute orders
    # -----------------------------
    def _execute_orders():
        with msg_output:
            clear_output()
            print("🚀 Placing all orders...\n")

        for order in orders:
            try:
                # Build order params
                order_params = {
                    "variety": order["variety"],
                    "exchange": order["exchange"],
                    "tradingsymbol": order["tradingsymbol"],
                    "transaction_type": order["transaction_type"],
                    "quantity": order["quantity"],
                    "order_type": order["order_type"],
                    "product": order["product"]
                }
                
                # Add price fields if they exist
                if 'price' in order:
                    order_params['price'] = order['price']
                if 'trigger_price' in order:
                    order_params['trigger_price'] = order['trigger_price']
                
                order_id = kite.place_order(**order_params)
                
                with msg_output:
                    display(HTML(f"<pre style='color: green; font-weight: bold'>✔ Order placed: {order['tradingsymbol']} → Order ID: {order_id}</pre>"))
            except Exception as e:
                with msg_output:
                    display(HTML(f"<pre style='color: red; font-weight: bold'>❌ Error placing {order['tradingsymbol']}: {e}</pre>"))

    # -----------------------------
    # Place Orders with insufficient funds confirmation
    # -----------------------------
    place_btn = widgets.Button(description="Place All Orders", button_style="success")

    def place_orders(b):
        if not orders:
            with msg_output:
                clear_output()
                print("⚠️ No orders to place!")
            return

        total_required, available_balance = check_margin_required()
        if total_required is None:
            return

        if total_required > available_balance:
            # Show confirmation Yes/No
            yes_btn = widgets.Button(description="Yes", button_style="success")
            no_btn = widgets.Button(description="No", button_style="danger")
            confirm_box = widgets.HBox([yes_btn, no_btn])

            with msg_output:
                print(f"⚠️ Insufficient funds! Required: ₹{total_required:,.2f}, Available: ₹{available_balance:,.2f}")
                print("Do you still want to proceed with placing orders?")
                display(confirm_box)

            def proceed_yes(b):
                confirm_box.close()
                _execute_orders()

            def proceed_no(b):
                with msg_output:
                    clear_output()
                    print("❌ Order placement cancelled.")

            yes_btn.on_click(proceed_yes)
            no_btn.on_click(proceed_no)
        else:
            _execute_orders()

    place_btn.on_click(place_orders)

    # -----------------------------
    # Layout FUTURES and OPTIONS
    # -----------------------------
    fut_box = widgets.VBox([
        widgets.HTML("<h4>FUTURES</h4>"),
        fut_widget["tradingsymbol"],
        fut_widget["side_box"],
        fut_widget["qty_box"],
        fut_widget["order_type"],
        fut_widget["price_container"],  # Dynamic price fields
        fut_widget["product"],
        fut_widget["variety"],
        fut_widget["add_btn"]
    ], layout=widgets.Layout(min_width='320px'))

    opt_box = widgets.VBox([
        widgets.HTML("<h4>OPTIONS</h4>"),
        opt_widget["tradingsymbol"],
        opt_widget["side_box"],
        opt_widget["qty_box"],
        opt_widget["order_type"],
        opt_widget["price_container"],  # Dynamic price fields
        opt_widget["product"],
        opt_widget["variety"],
        opt_widget["add_btn"]
    ], layout=widgets.Layout(min_width='320px'))

    container = widgets.HBox(
        [fut_box, opt_box],
        layout=widgets.Layout(
            justify_content='space-between',
            width='100%',
            padding='0 50px'
        )
    )

    # -----------------------------
    # Display everything
    # -----------------------------
    display(container)
    display(widgets.HBox([place_btn, clear_btn, margin_btn], layout=widgets.Layout(margin='20px 0 0 0')))
    display(basket_output)
    display(msg_output)


## Exit Positions

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

def exit_position_manager(kite):
    """
    Interactive position exit manager with multi-selection and customizable exit parameters.
    
    Args:
        kite: KiteConnect instance
    """
    
    # Constants
    EXECUTED_STATUSES = ['COMPLETE', 'TRIGGER PENDING', 'MODIFY AMO REQ RECEIVED', 'AMO REQ RECEIVED']
    REQUIRED_COLUMNS = ['status', 'tradingsymbol', 'order_type', 'variety', 'transaction_type', 
                       'product', 'quantity', 'price', 'average_price']
    
    # State
    selected_positions = []  # Now a list for multi-select
    exit_orders = []
    position_buttons = {}  # Store button references
    
    # Output widgets
    position_output = widgets.Output()
    exit_panel_output = widgets.Output()
    basket_output = widgets.Output()
    msg_output = widgets.Output()
    
    # ========================================
    # STEP 1: Fetch and Display Positions
    # ========================================
    def get_aggregated_positions():
        """Fetch orders and calculate aggregated positions"""
        try:
            orders_df = pd.DataFrame(kite.orders())
            
            if orders_df.empty:
                return None
            
            # Filter executed orders
            executed_orders = orders_df[orders_df['status'].isin(EXECUTED_STATUSES)][REQUIRED_COLUMNS].copy()
            
            if executed_orders.empty:
                return None
            
            # Group and aggregate
            grouped = executed_orders.groupby(
                ['tradingsymbol', 'order_type', 'transaction_type', 'product', 'variety'],
                as_index=False
            ).agg({
                'price': 'mean',
                'average_price': 'mean',
                'quantity': 'sum'
            })
            
            # Pivot to get BUY/SELL columns
            pivot_df = grouped.pivot_table(
                index=['tradingsymbol', 'order_type', 'product', 'variety', 'price'],
                columns='transaction_type',
                values='quantity',
                aggfunc='sum',
                fill_value=0
            ).reset_index()
            
            # Ensure BUY and SELL columns exist
            for col in ['BUY', 'SELL']:
                if col not in pivot_df.columns:
                    pivot_df[col] = 0
            
            # Calculate open positions
            pivot_df['OPEN_QTY'] = pivot_df['BUY'] - pivot_df['SELL']
            
            # Filter only open positions
            open_positions = pivot_df[pivot_df['OPEN_QTY'] != 0].copy()
            
            if open_positions.empty:
                return None
            
            # Calculate exit parameters
            open_positions['EXIT_QTY'] = open_positions['OPEN_QTY'].abs()
            open_positions['exit_type'] = open_positions['OPEN_QTY'].apply(
                lambda x: 'SELL' if x > 0 else 'BUY'
            )
            
            return open_positions
            
        except Exception as e:
            with msg_output:
                clear_output()
                print(f"❌ Error fetching positions: {e}")
            return None
    
    # ========================================
    # STEP 2: Display Position Buttons (Multi-select)
    # ========================================
    def display_position_buttons(positions_df):
        """Display positions as multi-selectable buttons"""
        nonlocal position_buttons
        position_buttons = {}
        
        with position_output:
            clear_output()
            display(HTML("<h3 style='color: #2196f3;'>📊 Active Positions - Select Positions to Exit</h3>"))
            # display(HTML("<p style='color: #666;'>Click positions to select/deselect. Selected positions will be highlighted.</p>"))
            
            buttons = []
            for idx, row in positions_df.iterrows():
                symbol = row['tradingsymbol']
                open_qty = row['OPEN_QTY']
                avg_price = row['price']
                side = 'LONG' if open_qty > 0 else 'SHORT'
                side_color = 'green' if open_qty > 0 else 'red'
                
                # Create button label
                label = f"{symbol} | {side} {abs(open_qty)} @ ₹{avg_price:.2f}"
                
                btn = widgets.Button(
                    description=label,
                    layout=widgets.Layout(width='auto', margin='5px'),
                    style={'button_color': None}
                )
                
                # Store position data in button
                btn.position_data = row.to_dict()
                btn.position_id = symbol  # Unique identifier
                btn.is_selected = False
                btn.on_click(lambda b: toggle_position_selection(b))
                
                buttons.append(btn)
                position_buttons[symbol] = btn
            
            # Display buttons in rows
            rows = [buttons[i:i+2] for i in range(0, len(buttons), 2)]
            for row in rows:
                display(widgets.HBox(row))
            
            # Action buttons
            action_box = widgets.HBox([
                widgets.Button(description="🔄 Refresh Positions", button_style="info"),
                widgets.Button(description="✅ Confirm Selection", button_style="success"),
                widgets.Button(description="❌ Clear Selection", button_style="warning")
            ], layout=widgets.Layout(margin='15px 0 0 0'))
            
            refresh_btn, confirm_btn, clear_sel_btn = action_box.children
            
            refresh_btn.on_click(lambda b: initialize_ui())
            confirm_btn.on_click(lambda b: show_multi_exit_panel())
            clear_sel_btn.on_click(lambda b: clear_all_selections())
            
            display(action_box)
    
    def toggle_position_selection(btn):
        """Toggle position selection state"""
        nonlocal selected_positions
        
        if btn.is_selected:
            # Deselect
            btn.is_selected = False
            btn.style.button_color = None
            selected_positions = [p for p in selected_positions if p['tradingsymbol'] != btn.position_id]
        else:
            # Select
            btn.is_selected = True
            btn.style.button_color = 'lightgreen'
            selected_positions.append(btn.position_data)
        
        # Show count of selected positions
        with msg_output:
            clear_output()
            if selected_positions:
                print(f"✓ Selected {len(selected_positions)} position(s): {', '.join([p['tradingsymbol'] for p in selected_positions])}")
            else:
                print("No positions selected")
    
    def clear_all_selections():
        """Clear all position selections"""
        nonlocal selected_positions
        selected_positions = []
        
        for btn in position_buttons.values():
            btn.is_selected = False
            btn.style.button_color = None
        
        with msg_output:
            clear_output()
            print("All selections cleared")
        
        with exit_panel_output:
            clear_output()
    
    # ========================================
    # STEP 3: Multi-Position Exit Panel
    # ========================================
    def show_multi_exit_panel():
        """Display exit panel for multiple selected positions"""
        if not selected_positions:
            with msg_output:
                clear_output()
                print("⚠️ Please select at least one position first!")
            return
        
        with exit_panel_output:
            clear_output()
            
            # Summary of selected positions
            # display(HTML(f"""
            #     <div style='background: #f5f5f5; padding: 15px; border-radius: 8px; margin: 20px 0;'>
            #         <h3 style='color: #2196f3; margin-top: 0;'>🎯 Exit {len(selected_positions)} Position(s)</h3>
            #     </div>
            # """))
            
            # Show each position with individual controls
            for idx, position in enumerate(selected_positions):
                create_position_exit_widget(position, idx)
    
    def create_position_exit_widget(position, index):
        """Create exit widget for a single position"""
        symbol = position['tradingsymbol']
        open_qty = position['OPEN_QTY']
        exit_qty = position['EXIT_QTY']
        exit_type = position['exit_type']
        avg_price = position['price']
        
        side = 'LONG' if open_qty > 0 else 'SHORT'
        side_color = 'green' if open_qty > 0 else 'red'
        
        # Position header
        display(HTML(f"""
            <div style='background: #e3f2fd; padding: 12px; border-radius: 5px; margin: 15px 0 10px 0; border-left: 4px solid {side_color};'>
                <strong style='font-size: 16px;'>{symbol}</strong> | 
                <span style='color: {side_color}; font-weight: bold;'>{side} {abs(open_qty)}</span> @ ₹{avg_price:.2f} | 
                Exit: <span style='color: {"red" if exit_type == "SELL" else "green"}; font-weight: bold;'>{exit_type}</span>
            </div>
        """))
        
        # Create widgets for this position
        quantity_input = widgets.IntText(
            value=int(exit_qty),
            description='Quantity:',
            layout=widgets.Layout(width='200px')
        )
        
        order_type = widgets.Dropdown(
            options=['MARKET', 'LIMIT', 'SL', 'SL-M'],
            value='SL-M',
            description='Order Type:',
            layout=widgets.Layout(width='200px')
        )
        
        price_input = widgets.FloatText(
            value=round(avg_price, 2),
            description='Price:',
            layout=widgets.Layout(width='200px')
        )
        
        trigger_price_input = widgets.FloatText(
            value=round(avg_price + 10, 2) if exit_type == 'SELL' else round(avg_price - 10, 2),
            description='Trigger:',
            layout=widgets.Layout(width='200px')
        )
        
        stoploss_points = widgets.FloatText(
            value=10.0,
            description='SL Points:',
            layout=widgets.Layout(width='150px', margin='0 5px 0 0')
        )
        
        calc_sl_btn = widgets.Button(
            description="Revise",
            button_style="info",
            layout=widgets.Layout(width='100px', margin='0')
        )
        
        def calculate_stoploss(b):
            sl_points = stoploss_points.value
            if exit_type == 'SELL':
                # For SELL exits (closing LONG), trigger below avg price
                trigger_price_input.value = round(avg_price - sl_points, 2)
            else:
                # For BUY exits (closing SHORT), trigger above avg price
                trigger_price_input.value = round(avg_price + sl_points, 2)
            # For SL orders, limit price is usually same as trigger or slightly better
            price_input.value = trigger_price_input.value
        
        calc_sl_btn.on_click(calculate_stoploss)
        
        product = widgets.Dropdown(
            options=['MIS', 'CNC', 'NRML'],
            value=position.get('product', 'MIS'),
            description='Product:',
            layout=widgets.Layout(width='200px')
        )
        
        variety = widgets.Dropdown(
            options=['regular', 'amo', 'co', 'iceberg'],
            value=position.get('variety', 'regular'),
            description='Variety:',
            layout=widgets.Layout(width='200px')
        )
        
        # Price container (dynamic visibility)
        price_container = widgets.VBox([])
        
        def update_price_fields(change):
            order_type_val = change['new']
            if order_type_val == 'MARKET':
                price_container.children = []
            elif order_type_val == 'LIMIT':
                price_container.children = [price_input]
            elif order_type_val in ['SL', 'SL-M']:
                sl_calc_box = widgets.HBox(
                    [stoploss_points, calc_sl_btn],
                    layout=widgets.Layout(margin='0 0 5px 0')
                )
                if order_type_val == 'SL':
                    price_container.children = [
                        sl_calc_box,
                        trigger_price_input,
                        price_input
                    ]
                else:
                    price_container.children = [
                        sl_calc_box,
                        trigger_price_input
                    ]
        
        order_type.observe(update_price_fields, names='value')
        update_price_fields({'new': order_type.value})
        
        # Layout controls in columns
        col1 = widgets.VBox([quantity_input, order_type, product])
        col2 = widgets.VBox([price_container, variety])
        
        control_box = widgets.HBox([col1, col2], layout=widgets.Layout(margin='0 0 10px 0'))
        display(control_box)
        
        # Add to basket button for this position
        add_btn = widgets.Button(
            description=f"➕ Add {symbol} to Basket",
            button_style="primary",
            layout=widgets.Layout(margin='0 0 15px 0')
        )
        
        def add_to_basket(b):
            order = build_order(position, quantity_input, order_type, price_input, 
                              trigger_price_input, product, variety, exit_type)
            exit_orders.append(order)
            display_basket()
            with msg_output:
                clear_output()
                print(f"✅ Added {symbol} exit order to basket")
        
        add_btn.on_click(add_to_basket)
        display(add_btn)
        
        # Divider
        if index < len(selected_positions) - 1:
            display(HTML("<hr style='border: none; border-top: 1px solid #ddd; margin: 20px 0;'>"))
    
    def build_order(position, qty_widget, order_type_widget, price_widget, 
                   trigger_widget, product_widget, variety_widget, exit_type):
        """Build order dictionary from widgets"""
        order = {
            'tradingsymbol': position['tradingsymbol'],
            'exchange': 'NFO',
            'transaction_type': exit_type,
            'quantity': int(qty_widget.value),
            'order_type': order_type_widget.value,
            'product': product_widget.value,
            'variety': variety_widget.value
        }
        
        # Add price fields based on order type
        if order_type_widget.value == 'LIMIT':
            order['price'] = float(price_widget.value)
        elif order_type_widget.value == 'SL':
            order['price'] = float(price_widget.value)
            order['trigger_price'] = float(trigger_widget.value)
        elif order_type_widget.value == 'SL-M':
            order['trigger_price'] = float(trigger_widget.value)
        
        return order
    
    # ========================================
    # Quick Add All to Basket
    # ========================================
    def add_all_to_basket_with_defaults():
        """Add all selected positions to basket with default SL settings"""
        if not selected_positions:
            with msg_output:
                clear_output()
                print("⚠️ No positions selected!")
            return
        
        for position in selected_positions:
            exit_type = position['exit_type']
            avg_price = position['price']
            
            # Default SL order
            order = {
                'tradingsymbol': position['tradingsymbol'],
                'exchange': 'NFO',
                'transaction_type': exit_type,
                'quantity': int(position['EXIT_QTY']),
                'order_type': 'SL',
                'product': position.get('product', 'MIS'),
                'variety': position.get('variety', 'regular'),
                'trigger_price': round(avg_price - 10, 2) if exit_type == 'SELL' else round(avg_price + 10, 2),
                'price': round(avg_price - 10, 2) if exit_type == 'SELL' else round(avg_price + 10, 2)
            }
            exit_orders.append(order)
        
        display_basket()
        with msg_output:
            clear_output()
            print(f"✅ Added {len(selected_positions)} positions to basket with default SL")
    
    # ========================================
    # STEP 4: Basket Management
    # ========================================
    def display_basket():
        """Display exit order basket"""
        with basket_output:
            clear_output()
            if not exit_orders:
                return
            
            display(HTML("<h3 style='color: #ff9800;'>🛒 Exit Order Basket</h3>"))
            
            for idx, order in enumerate(exit_orders):
                side_color = 'green' if order['transaction_type'] == 'BUY' else 'red'
                
                order_html = f"""
                    <div style='background: #f5f5f5; padding: 10px; margin: 5px 0; border-radius: 5px; border-left: 4px solid {side_color};'>
                        <strong>{order['tradingsymbol']}</strong> | 
                        <span style='color: {side_color}; font-weight: bold;'>{order['transaction_type']} {order['quantity']}</span> | 
                        {order['order_type']} | {order['product']}
                        {f" @ ₹{order.get('price', 0):.2f}" if 'price' in order else ""}
                        {f" (Trigger: ₹{order.get('trigger_price', 0):.2f})" if 'trigger_price' in order else ""}
                    </div>
                """
                display(HTML(order_html))
            
            # Basket actions
            clear_basket_btn = widgets.Button(
                description="🗑️ Clear Basket",
                button_style="danger",
                layout=widgets.Layout(margin='10px 5px 0 0')
            )
            
            place_all_btn = widgets.Button(
                description="🚀 Place All Orders",
                button_style="success",
                layout=widgets.Layout(margin='10px 0 0 0')
            )
            
            def clear_basket(b):
                exit_orders.clear()
                display_basket()
                with msg_output:
                    clear_output()
                    print("🗑️ Basket cleared")
            
            def place_all(b):
                place_all_orders()
            
            clear_basket_btn.on_click(clear_basket)
            place_all_btn.on_click(place_all)
            
            display(widgets.HBox([place_all_btn, clear_basket_btn]))
    
    # ========================================
    # STEP 5: Order Placement
    # ========================================
    def place_single_order(order):
        """Place a single order"""
        try:
            order_params = {k: v for k, v in order.items() if k != 'exchange'}
            order_params['exchange'] = order['exchange']
            order_params['validity'] = 'DAY'
            
            order_id = kite.place_order(**order_params)
            
            with msg_output:
                display(HTML(f"""
                    <div style='color: green; font-weight: bold; padding: 8px; background: #e8f5e9; border-radius: 5px; margin: 3px 0;'>
                        ✅ {order['tradingsymbol']}: Order ID {order_id}
                    </div>
                """))
        except Exception as e:
            with msg_output:
                display(HTML(f"""
                    <div style='color: red; font-weight: bold; padding: 8px; background: #ffebee; border-radius: 5px; margin: 3px 0;'>
                        ❌ {order['tradingsymbol']}: {str(e)}
                    </div>
                """))
    
    def place_all_orders():
        """Place all orders in basket"""
        if not exit_orders:
            with msg_output:
                clear_output()
                print("⚠️ No orders in basket!")
            return
        
        with msg_output:
            clear_output()
            print(f"🚀 Placing {len(exit_orders)} orders...\n")
        
        for order in exit_orders:
            place_single_order(order)
        
        exit_orders.clear()
        display_basket()
    
    # ========================================
    # Initialize UI
    # ========================================
    def initialize_ui():
        """Initialize the complete UI"""
        nonlocal selected_positions
        selected_positions = []
        
        positions_df = get_aggregated_positions()
        
        with position_output:
            clear_output()
            if positions_df is None:
                print("⚠️ No active positions found")
                return
        
        with exit_panel_output:
            clear_output()
        
        with msg_output:
            clear_output()
        
        display_position_buttons(positions_df)
    
    # Start the UI
    # display(HTML("<h2 style='color: #2196f3;'>📈 Multi-Position Exit Manager</h2>"))
    display(position_output)
    display(exit_panel_output)
    display(basket_output)
    display(msg_output)
    
    initialize_ui()


# Usage example:
# exit_position_manager(kite)